In [6]:
import os
import sys
print(os.getcwd())
print(os.path.dirname(os.path.realpath('__file__')))
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job
from acaisdk.meta import * 

from acaisdk.utils import utils
from acaisdk import credentials
# from acaisdk import automl
os.environ["CLUSTER"] = 'PHOEBE'
utils.DEBUG = True  # print debug messages
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory

/Users/nishantravishankar/Documents/Thesis/acaisdk/acaisdk/example/AdaptDL-Workloads
/Users/nishantravishankar/Documents/Thesis/acaisdk/acaisdk/example/AdaptDL-Workloads


In [13]:
def setup_workspace(project_id):
    os.environ["CLUSTER"] = 'PHOEBE'
    utils.DEBUG = True  # print debug messages
    credentials.login('sWEuAkD8NkLWXHRxhOWcbbesj6Ny1jBx')
    # Make your changes here
    root_token = 'EmDlCTBF1ppONSciYVd03M9xkmF6hFqW'
    project_admin = 'nravish2'
    user = 'nravish2'
    r = Project.create_project(project_id, root_token, project_admin, csp='AZURE')
    print(r)
    print(credentials.login(r['project_admin_token']))

In [9]:
def submit_adaptdl_job(workspace, name, description, input_file_set, application, target_num_replicas, target_batch_size):
    print(f"workspace = {workspace}, name = {name}, description = {description}, \
        input_file_set = {input_file_set}, application = {application}, \
        target_num_replicas = {target_num_replicas}, target_batch_size = {target_batch_size}")
    
    code = os.path.join(workspace, f'{application}.zip')
    
    File.upload({code: f'/{application}.zip'})
    print(f"Done uploading {application}.zip");
    print(File.list_dir('/'))
    download_dir = workspace + '/temp'
    File.download({f'/{application}.zip' : download_dir})
    print(f"Done verifying download")
    # Upload input files and create file set
    input_dir = os.path.join(workspace, 'data')
    
    File.convert_to_file_mapping([input_dir], 'data/') \
    .files_to_upload \
    .upload() \
    .as_new_file_set(input_file_set)

    print(f"Done uploading input_file_set. Now attempting to execute job")
    
    # Run a job
    job_setting = {
        "v_cpu": "",
        "memory": "",
        "gpu": "",
        "command": "",
        "container_image": "",
        'input_file_set': f'{input_file_set}:1',
        'output_path': '',
        'code': f'/{application}.zip',
        'description': description,
        'name': name,
        'adaptdl_status': 'Enabled',
        'application': application,
        'target_num_replicas': target_num_replicas,
        'target_batch_size': target_batch_size
    }
    print(f"job_setting = {job_setting}")
    j = Job().with_attributes(job_setting).run()
    print(j)


In [14]:
print("Setting up workspace")
project_id = "nravish2_adaptdl_10"
setup_workspace(project_id)

Setting up workspace
Set the budget of Project {nravish2_adaptdl_10} to default value: $10.00
Update the budget using: set_budget(amount)
Cloud service provider:  AZURE
phoebe-mgmt.pdl.local.cmu.edu cred_endpoint
30379 cred_port
Running request: phoebe-mgmt.pdl.local.cmu.edu 30379 credential create_project
POST data {"project_id": "nravish2_adaptdl_10", "budget": 10, "admin_token": "EmDlCTBF1ppONSciYVd03M9xkmF6hFqW", "csp": "AZURE", "project_admin_name": "nravish2"}
----------- printing post data -------------
phoebe-mgmt.pdl.local.cmu.edu
30379
{'project_admin_id': 146095, 'project_admin_token': 'kscGQOjkYG0WBKAkq1EuHP1YCycR0Mk4', 'bucket_name': ''}
None


In [ ]:
workload = pandas.read_csv("phoebe_sample_workload.csv")
start = time.time()
workspace = os.path.dirname(os.path.realpath('__file__'))  # get current directory
for row in workload.sort_values(by="time").itertuples():
    # while time.time() - start < row.time:
    #     time.sleep(1)
    description = "random"
    input_file_set = "data"
    print(row)
    #workspace, name, description, input_file_set, application, target_num_replicas, target_batch_size
    submit_adaptdl_job(workspace, row.application, description, 
                       "data", row.application,
                       row.num_replicas, row.batch_size)